In [41]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from cleaning import df
from first_steps import normalize

In [42]:
df

,player,href,height,season,age,tm,lg,pos,g,gs,...,orb,drb,trb,ast,stl,blk,tov,pf,pts,allstar
0,Kareem Abdul-Jabbar,/players/a/abdulka01.html,86,NaN,22,MIL,NBA,C,82.0,NaN,...,NaN,NaN,14.5,4.1,NaN,NaN,NaN,3.5,28.8,1
1,Kareem Abdul-Jabbar,/players/a/abdulka01.html,86,NaN,23,MIL,NBA,C,82.0,NaN,...,NaN,NaN,16.0,3.3,NaN,NaN,NaN,3.2,31.7,1
2,Kareem Abdul-Jabbar,/players/a/abdulka01.html,86,NaN,24,MIL,NBA,C,81.0,NaN,...,NaN,NaN,16.6,4.6,NaN,NaN,NaN,2.9,34.8,1
3,Kareem Abdul-Jabbar,/players/a/abdulka01.html,86,NaN,25,MIL,NBA,C,76.0,NaN,...,NaN,NaN,16.1,5.0,NaN,NaN,NaN,2.7,30.2,1
4,Kareem Abdul-Jabbar,/players/a/abdulka01.html,86,NaN,26,MIL,NBA,C,81.0,NaN,...,3.5,11.0,14.5,4.8,1.4,3.5,NaN,2.9,27.0,1
5,Kareem Abdul-Jabbar,/players/a/abdulka01.html,86,NaN,27,MIL,NBA,C,65.0,NaN,...,3.0,11.0,14.0,4.1,1.0,3.3,NaN,3.2,30.0,1
6,Kareem Abdul-Jabbar,/players/a/abdulka01.html,86,NaN,28,LAL,NBA,C,82.0,NaN,...,3.3,13.5,16.9,5.0,1.5,4.1,NaN,3.6,27.7,1
7,Kareem Abdul-Jabbar,/players/a/abdulka01.html,86,NaN,29,LAL,NBA,C,82.0,NaN,...,3.2,10.0,13.3,3.9,1.2,3.2,NaN,3.2,26.2,1
8,Kareem Abdul-Jabbar,/players/a/abdulka01.html,86,1977-78,30,LAL,NBA,C,62.0,NaN,...,3.0,9.9,12.9,4.3,1.7,3.0,3.4,2.9,25.8,0
9,Kareem Abdul-Jabbar,/players/a/abdulka01.html,86,NaN,31,LAL,NBA,C,80.0,NaN,...,2.6,10.2,12.8,5.4,1.0,4.0,3.5,2.9,23.8,1


In [43]:
unique = df['player'].unique()

In [44]:
for player in unique:
    print(df[df['player'] == player].player)
    

0     Kareem Abdul-Jabbar
1     Kareem Abdul-Jabbar
2     Kareem Abdul-Jabbar
3     Kareem Abdul-Jabbar
4     Kareem Abdul-Jabbar
5     Kareem Abdul-Jabbar
6     Kareem Abdul-Jabbar
7     Kareem Abdul-Jabbar
8     Kareem Abdul-Jabbar
9     Kareem Abdul-Jabbar
10    Kareem Abdul-Jabbar
11    Kareem Abdul-Jabbar
12    Kareem Abdul-Jabbar
13    Kareem Abdul-Jabbar
14    Kareem Abdul-Jabbar
15    Kareem Abdul-Jabbar
16    Kareem Abdul-Jabbar
17    Kareem Abdul-Jabbar
18    Kareem Abdul-Jabbar
19    Kareem Abdul-Jabbar
Name: player, dtype: object
20    Michael Jordan
21    Michael Jordan
22    Michael Jordan
23    Michael Jordan
24    Michael Jordan
25    Michael Jordan
26    Michael Jordan
27    Michael Jordan
28    Michael Jordan
29    Michael Jordan
30    Michael Jordan
31    Michael Jordan
32    Michael Jordan
33    Michael Jordan
34    Michael Jordan
35    Michael Jordan
36    Michael Jordan
37    Michael Jordan
38    Michael Jordan
Name: player, dtype: object
39    Bill Russell
40    

1389    Gail Goodrich
1390    Gail Goodrich
1391    Gail Goodrich
1392    Gail Goodrich
1393    Gail Goodrich
1394    Gail Goodrich
1395    Gail Goodrich
1396    Gail Goodrich
1397    Gail Goodrich
1398    Gail Goodrich
1399    Gail Goodrich
1400    Gail Goodrich
1401    Gail Goodrich
1402    Gail Goodrich
Name: player, dtype: object
1403    Amar'e Stoudemire
1404    Amar'e Stoudemire
1405    Amar'e Stoudemire
1406    Amar'e Stoudemire
1407    Amar'e Stoudemire
1408    Amar'e Stoudemire
1409    Amar'e Stoudemire
1410    Amar'e Stoudemire
1411    Amar'e Stoudemire
1412    Amar'e Stoudemire
1413    Amar'e Stoudemire
1414    Amar'e Stoudemire
1415    Amar'e Stoudemire
1416    Amar'e Stoudemire
Name: player, dtype: object
1417    Larry Costello
1418    Larry Costello
1419    Larry Costello
1420    Larry Costello
1421    Larry Costello
1422    Larry Costello
1423    Larry Costello
1424    Larry Costello
1425    Larry Costello
1426    Larry Costello
1427    Larry Costello
1428    Larry Coste

2426    Rasheed Wallace
2427    Rasheed Wallace
2428    Rasheed Wallace
2429    Rasheed Wallace
2430    Rasheed Wallace
2431    Rasheed Wallace
2432    Rasheed Wallace
2433    Rasheed Wallace
2434    Rasheed Wallace
2435    Rasheed Wallace
2436    Rasheed Wallace
2437    Rasheed Wallace
2438    Rasheed Wallace
2439    Rasheed Wallace
2440    Rasheed Wallace
2441    Rasheed Wallace
2442    Rasheed Wallace
2443    Rasheed Wallace
Name: player, dtype: object
2444    Giannis Antetokounmpo
2445    Giannis Antetokounmpo
2446    Giannis Antetokounmpo
2447    Giannis Antetokounmpo
2448    Giannis Antetokounmpo
2449    Giannis Antetokounmpo
Name: player, dtype: object
2450    Mark Aguirre
2451    Mark Aguirre
2452    Mark Aguirre
2453    Mark Aguirre
2454    Mark Aguirre
2455    Mark Aguirre
2456    Mark Aguirre
2457    Mark Aguirre
2458    Mark Aguirre
2459    Mark Aguirre
2460    Mark Aguirre
2461    Mark Aguirre
2462    Mark Aguirre
Name: player, dtype: object
2463    Buck Williams
2464    B

3557    Udonis Haslem
3558    Udonis Haslem
3559    Udonis Haslem
3560    Udonis Haslem
3561    Udonis Haslem
3562    Udonis Haslem
3563    Udonis Haslem
3564    Udonis Haslem
3565    Udonis Haslem
3566    Udonis Haslem
3567    Udonis Haslem
3568    Udonis Haslem
3569    Udonis Haslem
3570    Udonis Haslem
3571    Udonis Haslem
3572    Udonis Haslem
Name: player, dtype: object
3573    Harrison Barnes
3574    Harrison Barnes
3575    Harrison Barnes
3576    Harrison Barnes
3577    Harrison Barnes
3578    Harrison Barnes
3579    Harrison Barnes
Name: player, dtype: object
3580    Tyson Chandler
3581    Tyson Chandler
3582    Tyson Chandler
3583    Tyson Chandler
3584    Tyson Chandler
3585    Tyson Chandler
3586    Tyson Chandler
3587    Tyson Chandler
3588    Tyson Chandler
3589    Tyson Chandler
3590    Tyson Chandler
3591    Tyson Chandler
3592    Tyson Chandler
3593    Tyson Chandler
3594    Tyson Chandler
3595    Tyson Chandler
3596    Tyson Chandler
3597    Tyson Chandler
Name: play

In [91]:
all_norm_df = pd.DataFrame()
for player in unique:
    pnormdf = pd.DataFrame()
    playerdf = df[df['player'] == player]
    for col in playerdf:
        if playerdf[col].dtypes == np.float64:
            statdf = playerdf[col]
            try:
                normdf = pd.DataFrame({col:normalize(statdf)})
                pnormdf = pd.concat([pnormdf, normdf], axis=1)
            except:
                pass
    pnormdf['player'] = player
    all_norm_df = pd.concat([all_norm_df, pnormdf])

In [92]:
all_norm_df

,2p,2p%,2pa,3p,3p%,3pa,age,ast,blk,drb,...,g,gs,mp,orb,pf,player,pts,stl,tov,trb
0,0.712871,0.322835,0.825000,NaN,NaN,NaN,22.0,0.704545,NaN,NaN,...,1.000000,NaN,0.948357,NaN,0.900000,Kareem Abdul-Jabbar,0.757085,NaN,NaN,0.806452
1,0.871287,0.787402,0.850000,NaN,NaN,NaN,23.0,0.522727,NaN,NaN,...,1.000000,NaN,0.807512,NaN,0.600000,Kareem Abdul-Jabbar,0.874494,NaN,NaN,0.927419
2,1.000000,0.763780,1.000000,NaN,NaN,NaN,24.0,0.818182,NaN,NaN,...,0.950000,NaN,1.000000,NaN,0.300000,Kareem Abdul-Jabbar,1.000000,NaN,NaN,0.975806
3,0.861386,0.606299,0.900000,NaN,NaN,NaN,25.0,0.909091,NaN,NaN,...,0.700000,NaN,0.934272,NaN,0.100000,Kareem Abdul-Jabbar,0.813765,NaN,NaN,0.935484
4,0.742574,0.488189,0.800000,NaN,NaN,NaN,26.0,0.863636,NaN,NaN,...,0.950000,NaN,0.981221,NaN,0.300000,Kareem Abdul-Jabbar,0.684211,NaN,NaN,0.806452
5,0.821782,0.283465,0.968750,NaN,NaN,NaN,27.0,0.704545,NaN,NaN,...,0.150000,NaN,0.910798,NaN,0.600000,Kareem Abdul-Jabbar,0.805668,NaN,NaN,0.766129
6,0.683168,0.409449,0.762500,NaN,NaN,NaN,28.0,0.909091,NaN,NaN,...,1.000000,NaN,0.859155,NaN,1.000000,Kareem Abdul-Jabbar,0.712551,NaN,NaN,1.000000
7,0.653465,0.803150,0.612500,NaN,NaN,NaN,29.0,0.659091,NaN,NaN,...,1.000000,NaN,0.652582,NaN,0.600000,Kareem Abdul-Jabbar,0.651822,NaN,NaN,0.709677
8,0.643564,0.574803,0.656250,NaN,NaN,NaN,30.0,0.750000,NaN,NaN,...,0.000000,NaN,0.638498,NaN,0.300000,Kareem Abdul-Jabbar,0.635628,NaN,NaN,0.677419
9,0.544554,0.787402,0.493750,NaN,NaN,NaN,31.0,1.000000,NaN,NaN,...,0.900000,NaN,0.779343,NaN,0.300000,Kareem Abdul-Jabbar,0.554656,NaN,NaN,0.669355


In [47]:
for col in playerdf:
    if playerdf[col].dtypes == np.float64:
        print(col, df[col].dtypes)

g float64
gs float64
mp float64
fg float64
fga float64
fg% float64
3p float64
3pa float64
3p% float64
2p float64
2pa float64
2p% float64
efg% float64
ft float64
fta float64
ft% float64
orb float64
drb float64
trb float64
ast float64
stl float64
blk float64
tov float64
pf float64
pts float64


In [85]:
columns = list(all_norm_df.columns)

In [86]:
top = columns.pop(21)
columns = [top] + columns

In [87]:
columns

['player',
 '2p',
 '2p%',
 '2pa',
 '3p',
 '3p%',
 '3pa',
 'ast',
 'blk',
 'drb',
 'efg%',
 'fg',
 'fg%',
 'fga',
 'ft',
 'ft%',
 'fta',
 'g',
 'gs',
 'mp',
 'orb',
 'pf',
 'pts',
 'stl',
 'tov',
 'trb']

In [88]:
all_norm_df = all_norm_df[columns]

In [89]:
all_norm_df

,player,2p,2p%,2pa,3p,3p%,3pa,ast,blk,drb,...,fta,g,gs,mp,orb,pf,pts,stl,tov,trb
0,Kareem Abdul-Jabbar,0.712871,0.322835,0.825000,NaN,NaN,NaN,0.704545,NaN,NaN,...,1.000000,1.000000,NaN,0.948357,NaN,0.900000,0.757085,NaN,NaN,0.806452
1,Kareem Abdul-Jabbar,0.871287,0.787402,0.850000,NaN,NaN,NaN,0.522727,NaN,NaN,...,0.884058,1.000000,NaN,0.807512,NaN,0.600000,0.874494,NaN,NaN,0.927419
2,Kareem Abdul-Jabbar,1.000000,0.763780,1.000000,NaN,NaN,NaN,0.818182,NaN,NaN,...,0.985507,0.950000,NaN,1.000000,NaN,0.300000,1.000000,NaN,NaN,0.975806
3,Kareem Abdul-Jabbar,0.861386,0.606299,0.900000,NaN,NaN,NaN,0.909091,NaN,NaN,...,0.565217,0.700000,NaN,0.934272,NaN,0.100000,0.813765,NaN,NaN,0.935484
4,Kareem Abdul-Jabbar,0.742574,0.488189,0.800000,NaN,NaN,NaN,0.863636,NaN,NaN,...,0.434783,0.950000,NaN,0.981221,NaN,0.300000,0.684211,NaN,NaN,0.806452
5,Kareem Abdul-Jabbar,0.821782,0.283465,0.968750,NaN,NaN,NaN,0.704545,NaN,NaN,...,0.637681,0.150000,NaN,0.910798,NaN,0.600000,0.805668,NaN,NaN,0.766129
6,Kareem Abdul-Jabbar,0.683168,0.409449,0.762500,NaN,NaN,NaN,0.909091,NaN,NaN,...,0.811594,1.000000,NaN,0.859155,NaN,1.000000,0.712551,NaN,NaN,1.000000
7,Kareem Abdul-Jabbar,0.653465,0.803150,0.612500,NaN,NaN,NaN,0.659091,NaN,NaN,...,0.623188,1.000000,NaN,0.652582,NaN,0.600000,0.651822,NaN,NaN,0.709677
8,Kareem Abdul-Jabbar,0.643564,0.574803,0.656250,NaN,NaN,NaN,0.750000,NaN,NaN,...,0.492754,0.000000,NaN,0.638498,NaN,0.300000,0.635628,NaN,NaN,0.677419
9,Kareem Abdul-Jabbar,0.544554,0.787402,0.493750,NaN,NaN,NaN,1.000000,NaN,NaN,...,0.536232,0.900000,NaN,0.779343,NaN,0.300000,0.554656,NaN,NaN,0.669355


In [ ]:
#Example of working through a player's normalized data
allen = df.loc[df.player == 'Allen Iverson']
alnorm = pd.DataFrame()
for col in allen.columns:
    try:
        alnorm[col] = normalize(allen[col])
    except:
        print(col)
for i, row in alnorm.iterrows():
    print(i, sum(row))

mj = df.loc[df.player == 'Michael Jordan']
mjnorm = pd.DataFrame()
for col in mj.columns:
    try:
        mjnorm[col] = normalize(mj[col])
    except:
        print(col)